In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import datetime
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser3"
password = "hello"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#image solution: https://community.plotly.com/t/adding-local-image/4896
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Center(html.B(html.H1('Mark Webster', style={'color': 'red'}))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()))), #GS logo
    html.Hr(),
    
    html.Div([
        html.Center(html.Label(['Select an option to filter the table by rescue type:'], style={'font-weight': 'bold'})),
        html.Center(dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'Filter by Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Filter by Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking Rescue', 'value': 'Filter by Disaster or Individual Tracking Rescue'},
                {'label': 'Reset', 'value': 'Remove all filters'}
            ],
            value='Filter table by rescue dog types',
            inputStyle={"margin-left": "30px"},
            labelStyle={'display': 'inline-block'}
        ))
    ]),
    html.Br(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single", #sets it so we can select one row at a time (so our map can update)
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                        dcc.Graph(
                             id='pie-chart-id',
                        ),
                        html.Div(
                             id='map-id',
                             className='col s12 m6',
                        )
                     ]
                 ),
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if selected_rows is None:
        return dash.no_update
    return [{
        'if': { 'row_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]



                                                 
@app.callback(
     Output('map-id', "children"),  #outputs according to the div in layout
    [Input('datatable-id', "derived_viewport_data"),  # takes in the "view" data which is the table with or without filtering
     Input('datatable-id', "derived_virtual_selected_rows"), # passes in which row we selected
     ])

def update_map(viewData, derived_virtual_selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    
    dff = df if viewData is None else pd.DataFrame(viewData)
    
    selectedAnimal = None
    
    if not derived_virtual_selected_rows:  #if there are not selected rows
        selectedAnimal = dff.iloc[0]
    else:
        selectedAnimal = dff.iloc[derived_virtual_selected_rows[0]] #else show based on the dataframe of the selected row
        
        #get data from dataframe locations
    latCoord = selectedAnimal[13]
    longCoord = selectedAnimal[14]
    animalBreed = selectedAnimal[4]
    animalName = selectedAnimal[9]
    
    return [
        dl.Map(style={'width': '700px', 'height': '500px'}, center=[latCoord, longCoord], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latCoord, longCoord], children=[ #place marker at coords
                dl.Tooltip(animalBreed), #hover shows breed
                dl.Popup([ #clicking on marker show name
                    html.H1("Animal Name"),
                    html.P(animalName)
                ])
            ])
        ])
    ]

@app.callback(
    Output('pie-chart-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graph(viewData):
    """ functionality for pie chart """
    dff = pd.DataFrame(viewData)

    piechart = px.pie(
        data_frame=dff,
        # pie chart measures outcome_type column (adopted, transfer, etc)
        names=dff['outcome_type'],
    )
    return piechart

@app.callback(
    Output('datatable-id', "data"),
    [Input('radio-items', "value")])
def radioItemsFilter(radioData):
    if radioData == 'Filter by Water Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'})) #only shows dogs
        dff = dff.loc[(dff['breed'] == 'Labrador Retriever Mix') | #    filters by breed
                      (dff['breed'] == 'Chesapeake Bay Retriever') |
                      (dff['breed'] == 'Newfoundland')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Female')] #further filters by sex
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156') # further fiters by age

    elif radioData == 'Filter by Mountain or Wilderness Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Alaskan Malamute') |
                      (dff['breed'] == 'Old English Sheepdog') |
                      (dff['breed'] == 'Siberian Husky') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')

    elif radioData == 'Filter by Disaster or Individual Tracking Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Doberman Pinscher') |
                      (dff['breed'] == 'Golden Retriever') |
                      (dff['breed'] == 'Bloodhound') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('20 <= age_upon_outcome_in_weeks <= 300')

    else:
        dff = pd.DataFrame.from_records(shelter.read({})) #else, maintains default table view

    return dff.to_dict('records') #returns new datatable
        
app